# Table of Contents
* [Learning Objectives:](#Learning-Objectives:)
* [Plot Tools](#Plot-Tools)
	* [Selection Tools](#Selection-Tools)
	* [Custom Plot Tools](#Custom-Plot-Tools)
	* [HoverTool](#HoverTool)
		* [DataFrame](#DataFrame)
		* [ColumnDataSource](#ColumnDataSource)
	* [Taptool](#Taptool)


# Learning Objectives:

After completion of this module, learners should be able to:

* customize `bokeh` interactivity tools in Figures
* plot data in Pandas DataFrames and ColumnDataSource objects

The raw data sets used in this noteook are stored in the `data/bokeh/*` folder in this repository along with importable Python scripts to read the CSV or JSON files into Pandas DataFrames.

Other topics not covered here
* [Widgets](http://bokeh.pydata.org/en/latest/docs/user_guide/interaction.html#adding-widgets)    
* [Javascript Callbacks](http://bokeh.pydata.org/en/latest/docs/user_guide/interaction.html#defining-callbacks)
    * [Callback example](http://nbviewer.ipython.org/github/bokeh/bokeh-notebooks/blob/master/tutorial/A1%20-%20Building%20gapminder.ipynb)

In [ ]:
from bokeh.io import output_notebook, show
output_notebook()

# Plot Tools

A number of Plot Tools are available that allow gesture-based interactivity with Bokeh plots.

By default all plots have the following Tools enabled
* Pan / Drag
* Box Zoom
* Resize
* Wheel Zoom
* Preview / Save
* Reset
* Help

In the `figure` method the default `tools` argument is `'pan,wheel_zoom,box_zoom,save,resize,reset,help'`

Custom tool selctions can be provided as a comma separated list of string to the `tools` keyword argument to figure() and chart functions.

The [Bokeh User Guide](http://bokeh.pydata.org/en/latest/docs/user_guide/tools.html) is very thorough on the usage of Tools in figures and charts.

Here's a list of other tools that can be configured in a plot

* Pan / Drag Tools
    * `box_select`
    * `box_zoom`
    * `lasso_select`
    * `pan`
    * `resize`
* Click / Tap Tools
    * `poly_select`
    * `tap_select`
* Scroll / Pinch Tools
    * `wheel_zoom`
* Actions
    * `reset`
    * `save`
* Inspectors
    * `crosshair`
    * `hover`

In [ ]:
import numpy as np

x = np.linspace(0, 4*np.pi, 100)
y = np.sin(x)

## Selection Tools

Two selection tools are available `box_select` and `lasso_select`. When using a glpyh other than `line` the selection tools are automatically set to dim unselected data points. Selecting a region without data points will reset the selection.

In [ ]:
from bokeh.plotting import figure

plot = figure(tools='reset,box_select,lasso_select,help')
plot.circle(x, y, color='blue')
show(plot)

## Custom Plot Tools

Plot Tools are their own classes (CamelCase naming convention with 'Tool' at the end) and can be customized by importing the tools from `bokeh.models`, creating new objects using the `add_tools` to associate them with the figure.

In [ ]:
from bokeh.models import BoxZoomTool,ResetTool
plot = figure(tools='')
plot.line(x, y, color='blue', line_width=2)
plot.add_tools(BoxZoomTool(dimensions=["width"]))
plot.add_tools(ResetTool())
show(plot)

## HoverTool

### DataFrame

The default hover tool will display the index of the nearest data point along with the position of the curser in Data (x,y) space and Canvas space.

In this example we will import the flower morphology DataFrame and use the lower-level plotting interface.

Notice that creating a legend is not automatic as it was with the Chart interface.

In [ ]:
from data.bokeh.iris import flowers
flowers

In [ ]:
from bokeh.models import BoxZoomTool,ResetTool,HoverTool

## Add a new Series mapping the species to a color
colormap = {'setosa': 'red', 'versicolor': 'green', 'virginica': 'blue'}
flowers['color'] = flowers['species'].map(lambda x: colormap[x])

tools = [BoxZoomTool(),ResetTool(),HoverTool()]

plot = figure(title = "Iris Morphology", tools=tools)
plot.xaxis.axis_label = 'Petal Length'
plot.yaxis.axis_label = 'Petal Width'

plot.circle(
    flowers["petal_length"], 
    flowers["petal_width"],
    color=flowers["color"], #assign the color to each circle
    fill_alpha=0.2, size=10 )

show(plot)

### ColumnDataSource

To provide more control over what information can be presented in a hover tooltip we introduce `ColumnDataSource` from `bokeh.plotting`. This special dictionary interacts with the `source` keyword argument in the glyph methods to provide efficient key-based access for the arguments. In addition the `source` data structure is accessible in the `HoverTool`.

In [ ]:
from bokeh.plotting import ColumnDataSource

#generate the source data from the DataFrame
sourceData = ColumnDataSource(
  data = dict(
      x=flowers['petal_length'],
      y=flowers['petal_width'],
      color=flowers['color'],
      species=flowers['species'],
      swidth=flowers['sepal_width'],
      slength=flowers['sepal_length']
    )
)

#specify what to display in the tooltip
hover = HoverTool(
  tooltips = [
        ('species','@species'),
        ('sepal width','@swidth'),
        ('sepal length','@slength')
    ]
)

tools = [BoxZoomTool(),ResetTool(),hover]

plot = figure(title = "Iris Morphology", tools=tools)
plot.xaxis.axis_label = 'Petal Length'
plot.yaxis.axis_label = 'Petal Width'

plot.circle(
    x='x',
    y='y',
    color='color',
    source=sourceData,
    fill_alpha=0.2, size=10 )

show(plot)

## Taptool

Sometimes we may want a tap event on a figure to open browser at a given URL
TapTool uses the same @ notation to form a URL that is specific to the data being touched

In [ ]:
from bokeh.models import ColumnDataSource, OpenURL, TapTool
from bokeh.plotting import figure, output_file, show

output_file("tmp/openurl.html")

p = figure(plot_width=400, plot_height=400,
           tools="tap", title="Click the Dots")

source = ColumnDataSource(data=dict(
    x=[1, 2, 3, 4, 5],
    y=[2, 5, 8, 2, 7],
    color=["navy", "orange", "olive", "firebrick", "gold"]
    ))

p.circle('x', 'y', color='color', size=20, source=source)

url = "http://www.colors.commutercreative.com/@color/"
taptool = p.select(type=TapTool)
taptool.callback = OpenURL(url=url)

show(p)